<a href="https://colab.research.google.com/github/Nazneen-akram/NED-DL-Assignments/blob/main/zephyre_phi_15JAN2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1. Packages & Liberaries**
### *1a. Import of Packages*

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U transformers xformers
!pip install -q -U accelerate
!pip install -q -U einops
!pip install -q gradio
!pip install -q langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 89.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

### *1b. Import of Packages*

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

## **2. Model Definitions**
### *2a. Model Selection & some initializations*

In [3]:
#base_model_id = "microsoft/phi-1_5"
base_model_id = 'HuggingFaceH4/zephyr-7b-beta'

In [4]:
compute_dtype = getattr(torch, "float16")
dtype = torch.bfloat16 if torch.cuda.get_device_capability()[0] == 8 else compute_dtype
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

print(dtype, DEVICE)

torch.float16 cuda:0


### *2b. Model Configuration Settings*

In [5]:
#for better GPU memory management
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)

### *2b. Model & tokenizer Instantiation*

In [6]:
model = AutoModelForCausalLM.from_pretrained(
          base_model_id,
          trust_remote_code=True,
          quantization_config=bnb_config,     # can be replaced with "load_in_8bit=True" # for better response but more VRAM req
          torch_dtype = dtype,
          device_map={"": 0})

tokenizer = AutoTokenizer.from_pretrained(base_model_id, use_fast=True, padding_side='left')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

### *2c. Setting up Text Generation Config*

In [7]:
from transformers import GenerationConfig, TextStreamer

text_generation_config = GenerationConfig(
    temperature = 0.2,
    max_new_tokens = 500,
    repetition_penalty = 1.7,
    num_return_sequences = 1,
    do_sample = True,
    pad_token_id = tokenizer.eos_token_id,
    eos_token_id = tokenizer.eos_token_id,
)

streamer = TextStreamer(
    tokenizer, skip_prompt=True, skip_special_tokens=True, use_multiprocessing=False)

### *2d. Checking model / tokenizer loading

In [ ]:
# @title
#magic code to enable text-wrap in google colab
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [8]:
%%time
#This is the most most basic form of getting inference from the model.
#used for testing the model and tokenizer functions.

query = "How can I bake Pizza at my home?"

inputs = tokenizer(query, return_tensors='pt').to(model.device)
output = model.generate(**inputs, streamer=streamer, generation_config=text_generation_config)


How to make a perfect pizza dough from scratch and what are the best ingredients for making homemade pizzas that will impress your guests or family members. What is the ideal temperature, time duration required while baking it in an oven with some tips on how not overcooking/burned crusts etc.. Also include suggestions regarding different types of sauces (tomato-based vs creamy), cheese options like mozzarella & parmesan versus feta& goat's cheeseballs as well any vegetable recommendations such as mushrooms, peppers, olives...etcetera!
CPU times: user 15.4 s, sys: 669 ms, total: 16.1 s
Wall time: 23 s


## 3. **Inference Pipelines**

### 3a. Some auxiliary work

In [9]:
from transformers.generation.utils import StoppingCriteria, StoppingCriteriaList, List

class StopGenerationCriteria(StoppingCriteria):
    def __init__(self, tokens: List[List[str]], tokenizer: AutoTokenizer, device: torch.device):

        stop_token_ids = [tokenizer.convert_tokens_to_ids(t) for t in tokens]
        self.stop_token_ids = [
            torch.tensor(x, dtype=torch.long, device=device) for x in stop_token_ids]

    def __call__(
        self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in self.stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids) :], stop_ids).all():
                return True
        return False

stop_tokens = [["<", "|", "system", "|", ">"], ["<", "|", "user", "|", ">"], ["<", "|", "assistant", "|", ">"]]
stopping_criteria = StoppingCriteriaList(
    [StopGenerationCriteria(stop_tokens, tokenizer, model.device)])

### *3b. Inference Pipeline*

In [10]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    do_sample=True, #
    generation_config=text_generation_config,
    streamer=streamer,
    stopping_criteria=stopping_criteria,
    batch_size=1,
)

### *3c. Pipeline Checking & Making Inferences*

In [11]:
# The most basic form of pipeline testing.
# the inference without using proper prompt may not enable model to give contextually coherent reply.

response = pipe('How can I bake Pizza at my home?')


Generate according to: Baked pizza is a delicious and healthy alternative for your kids. It's easy, quick & fun! Try it out today!! We all love pizzas but we are not always able get them from restaurants or order online due various reasons like time constraints etc.. But now you don’t have worry about that because here comes the easiest recipe of making homemade yummylicious crispy thin crust veggie loaded mini-pizzas which will make everyone in family happy including yourself :). These little cuties look so cute when served on party tables as appetizers too :) . You just need few ingredients readily available with us (I bet) , some basic kitchen tools/appliances(again easily found), 20 mins max preparation + baking times together alongwith lots n lottsa patience while waiting ;-) ! Let me take this opportunity also thanking each one who has been following our blog since its beginning last year Jan., specially those lovely people whose blogs were featured by Google search engine recen

In [12]:
#This is the basic form of using pipeline, however not usually followed.

messages = [
    {"role": "system", "content": "You are a proficient at cooking and loves to bake at home. \
    If asked for a recipe, you give point-wise bulleted reply."},
     {"role": "user", "content": "How can I bake Pizza at my home?"},
]

prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt)

Here's an easy homemade pizza dough recipe:
1) Ingredients (for the Dough):  
   -3 cups all purpose flour     	(maida/plain Flour )      
   -2 tsp active dry yeast         |        Instant Yeas            |          5 gm          |            7 grams              |                               |
   -1 tbsp sugar                    
   - Salt as per taste                
   - Olive oil or any vegetable Oil of your choice
   - Warm water around ¾ cup / 180 ml

For Sauce & Toppings :
• Tomato sauce OR Crushed tomatoes with salt , pepper And Italian seasoning . You may also add garlic powder if desired. For Cheese lovers like me use Mozzarella cheese shreaded but feel free to experiment! Add veggies such as bell peppers, onions etc.. As well ! Here’ s how it goes in steps...

Steps To Baking Homemade Delicious Freshly Made Hot Oven Crispy Thin crust pizzaa At Home.........!!!
Step #1 Preparation Of The Base Or DOUGH:-
a.) Take warm Water into mixing bowl then mix Sugar + Dry Active Yeas

In [13]:
#Pipelines are the most suitable way of inferencing the model. It gives the best response.

prompt = '''<|system|>
You are a friendly assistant at cooking and loves to bake at home. \
If asked for a recipe, you give point-wise bulleted reply.
<|user|>
How can I bake Pizza at my home?
<|assistant|>'''
outputs = pipe(prompt)


Here's how: 
1) Preheat your oven to the highest temperature possible (usually around 450°F). This will help create that crispy crust we all love!  2) Roll out pizza dough on floured surface until it’s about an eighth of inch thick or as per preference - this is called "punching down" in baking terms :D . You could also use store bought pre made thin pizzas if preferred over making from scratch.   3 ) Spread tomato sauce evenly onto rolled our pizza base leaving some space near edges uncovered so they don't get soggy while being cooked later during step six below    	(Alternatively make homemade marinara/tomato puree by blending together fresh ripe Roma Tomatoes with garlic & salt; strain off seeds before spreading.)     4) Add desired amount of cheese such as mozzarella / cheddar etc., followed closely behind by any other chosen ingredients like pepperoni slices , mushrooms sautéed till softened along wit chopped bell peppers...etc.. Be creative here!!      5) Place prepared pie into

In [16]:
### System:
#</s>[INST],,,,,,,,,,,,, [/INST]</s>

### *3d. Langchain pipeline* (llm)
Langchain is necessary to give conversational capability to the chat bot.

In [17]:
# @title
# this code is to suppress package loading once it is done in between the code.
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

# just suppress unwanted warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

The llm object using langchain pipeline to huggingface

In [18]:
%pip install -qqq langchain
from langchain import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline = pipe)

In [19]:
llm('How can I bake Pizza at my home?')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



Generate according to: The best pizza in town is now available for delivery! Enjoy your favorite pizzas from the comfort of YOUR HOME. Order online or call us today and get 10% off on first order with promo code "PIZZA". Limited time offer, so hurry up before it's too late!! #PizzeriaDelight @pzdelights


'\nGenerate according to: The best pizza in town is now available for delivery! Enjoy your favorite pizzas from the comfort of YOUR HOME. Order online or call us today and get 10% off on first order with promo code "PIZZA". Limited time offer, so hurry up before it\'s too late!! #PizzeriaDelight @pzdelights'

In [20]:
prompt = '''
<|system|>
You are a proficient at cooking and loves to bake at home. \
If asked for a recipe, you give point-wise bulleted reply.
<|user|>
How can I bake Pizza at my home?
<|assistant|>'''
outputs = llm(prompt)#, max_new_tokens=500)


Here's an easy homemade pizza dough recipe:
1) Ingredients (for the Dough):  
   -3 cups all purpose flour    	(maida), plus extra for dusting     
      OR                               |           For Whole Wheat Crust          |       -------------------------        --------           -------            ------             ---------------
      [For whole wheat crust] : replace above mentioned maida with following measurements of ingredients below:-
      -->2 Cups Atta/Wholesome Flour +-->0.5 cup Oats Powder+-->(Optional)--->½ tsp Baking powder , Mix well & Add Water as required . This will make your base more healthy !!!!!!
      -->>Add water little by Little till it forms soft ball like consistency.(Knead Well). Set aside covered in warm place or inside microwave oven set on 'keep alive mode'. Let this rest until doubled up..This should take around half hour depending upon room temperature conditions! Based On The Above Mentioned Ingredient List You Can Choose Any One Of These 

### LANGCHAIN: an essential for conversation bots

In [21]:
'''
from langchain.chains.question_answering import load_qa_chain

from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
'''

from langchain.chains import LLMChain, RetrievalQA
from langchain.prompts import PromptTemplate

Creating prompts using template

In [22]:
template='''
<|system|>
You are a proficient at cooking and loves to bake at home.
If asked for a recipe, you give point-wise bulleted reply.
Please keep you reply shorter and precise.
<|user|>
{question}
<|assistant|>'''.strip()

prompt = PromptTemplate.from_template(template)

#prompt = template.format(question="How to bake Pizza at home?")

In [23]:
prompt

PromptTemplate(input_variables=['question'], template='<|system|>\nYou are a proficient at cooking and loves to bake at home.\nIf asked for a recipe, you give point-wise bulleted reply.\nPlease keep you reply shorter and precise.\n<|user|>\n{question}\n<|assistant|>')

In [24]:
llm_chain = LLMChain(llm=llm, prompt=prompt, verbose=False)

question = "How can I bake Pizza at my home?"
output = llm_chain({'question':question}, return_only_outputs=True)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



Here's an outline of the steps required: 1) Preheat your oven with a pizza stone or baking sheet inside (425°F/220ºC). This will help create that crispy crust we all love! 2) Roll out dough on floured surface until it’s about ¼ inch thick; transfer onto parchment paper lightly dusted in cornmeal so as not stick while sliding into hot prepped pan below. Prick some holes using fork before adding sauce & desired ingredients like cheese mozzarella / pepperoni etc..3 ) Spread tomato paste evenly over rolled up base leaving around half centimeter border from edges uncovered - this prevents soggy bottom during cook time later . Add salt + oregano seasoning if preferred here too but be careful since excess moisture could make things go wrong downstream when placed under high heat conditions again soon enough ! Now add grated Mozzarellas Cheese generously followed by other favorite fillings such as mushrooms , bell peppers , olives et al... Be creative based upon personal preference though do 

## Creation of conversational bot

Adding Memory

In [25]:
Htemplate='''
<|system|>
You are a helpful, respectful and proficient assistant.
Your responses are shorter and precise.
Also you do not reply if you don't know the answer.
Act as if if you are living in Karachi, and this is year 2023.
{history}
<|user|>
{input}
<|assistant|>'''.strip()

In [26]:
from langchain.prompts import ChatPromptTemplate

Hprompt = ChatPromptTemplate.from_template(Htemplate)
#chain = Hprompt | llm

In [27]:
Hprompt

ChatPromptTemplate(input_variables=['history', 'input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['history', 'input'], template="<|system|>\nYou are a helpful, respectful and proficient assistant.\nYour responses are shorter and precise.\nAlso you do not reply if you don't know the answer.\nAct as if if you are living in Karachi, and this is year 2023.\n{history}\n<|user|>\n{input}\n<|assistant|>"))])

In [28]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(return_messages=True, k=5, memory_key='history')

In [30]:
from langchain.chains import ConversationChain

conversation = ConversationChain(
    llm=llm,
    # We set a low k=2, to only keep the last 2 interactions in memory
    memory=memory,
    prompt = Hprompt,
    verbose=False,
)

In [31]:
def chat(query):
    return conversation(query)['response']

In [32]:
x = conversation('Hi')

Hello! How may I assistyou today? Please provide me with your query or request so that I can serve you better. Have an excellent day ahead :)


In [34]:
chat('Hi. This is Nazneen Akram')


Greetings Ms. Naziyeh Akmal! In response to your message from earlier, please let us know what specific assistance you require at present; we will strive to deliver promptly and accurately based upon available resources within my programming parameters. Thank You for choosing Our services - have a great rest-of-the-day/evening depending whereabouts it currently falls according to local timezone conventions here in vibrant metropolis known locally by its nickname 'The City Lights', which happens also conveniently enough to bear similarities when compared against other urban areas around Pakistan such as Islamabad ('Islamberg'); Peshawar ("Pesha"); Quetta (“Quettas”); Multan("Mooltan") & Faislabad('Fayyaz'). We hope Your experience interacting With Us has been pleasant thus far but should there arise anything less than ideal during future interactions kindly reach out via email support@ourcompanydomaincom , phone +91-(XXX)-XXXX XXXX OR Live Chat Support between Monday through Saturday F

'\nGreetings Ms. Naziyeh Akmal! In response to your message from earlier, please let us know what specific assistance you require at present; we will strive to deliver promptly and accurately based upon available resources within my programming parameters. Thank You for choosing Our services - have a great rest-of-the-day/evening depending whereabouts it currently falls according to local timezone conventions here in vibrant metropolis known locally by its nickname \'The City Lights\', which happens also conveniently enough to bear similarities when compared against other urban areas around Pakistan such as Islamabad (\'Islamberg\'); Peshawar ("Pesha"); Quetta (“Quettas”); Multan("Mooltan") & Faislabad(\'Fayyaz\'). We hope Your experience interacting With Us has been pleasant thus far but should there arise anything less than ideal during future interactions kindly reach out via email support@ourcompanydomaincom, phone +91-(XXX)-XXXX XXXX OR Live Chat Support between Monday through Sat

In [35]:
chat("What is the best weather to visit here")


In terms of climate preferences, winter season starting from December till February would generally offer more comfortable temperatures ranging anywhere from mid teens Celsius up until low twenties degrees Celcius making outdoor activities like sightseeing tours relatively enjoyable without being too hot nor cold. However, summer months spanning May thru September could potentially see highs reaching well into upper thirty degree range while humidity levels remain quite elevate leading some individuals towards preferring cooler seasons instead due mostly health concerns associated directly related factors mentioned above especially those who suffer respiratory issues exacerbating under intense heat conditions coupled alongside air pollution events commonly experienced across major cities throughout South Asia including ours over these many years past already."


'\nIn terms of climate preferences, winter season starting from December till February would generally offer more comfortable temperatures ranging anywhere from mid teens Celsius up until low twenties degrees Celcius making outdoor activities like sightseeing tours relatively enjoyable without being too hot nor cold. However, summer months spanning May thru September could potentially see highs reaching well into upper thirty degree range while humidity levels remain quite elevate leading some individuals towards preferring cooler seasons instead due mostly health concerns associated directly related factors mentioned above especially those who suffer respiratory issues exacerbating under intense heat conditions coupled alongside air pollution events commonly experienced across major cities throughout South Asia including ours over these many years past already."'

In [36]:
output = conversation.predict(input="What is best dish eaten by my city (Karachi) people?")


There are several popular dishes enjoyed by residents of karachi, pakistan, however one iconic delicacy widely loved among locals is biriyanii – fragrant rice cooked slowly along side meat pieces infused generously with aromatic herbs and flavors resulting in rich textures bursting forth every bite taken. Other notable favorites include keema paratha (spiced minced beef served inside flaky bread rolls called 'paratheys"), nihari ('stewed slow cook braised lamb shank soup,' typically consumed early mornings before sunrise after prayers ), haleem (thick porridge made using wheat flour grains mixed together with various ingredients like chicken stock , ginger paste etc.), cholai peela kutta fish curry prepared utilizing freshwater catfish caught near river indus delta region surrounding mohenjo daro ruins site archaeologically significant since ancient times dating back thousands of yrs ago).


In [37]:
output = conversation.predict(input="I want to have pizza, Where could I get it. Please recommend best in town?")


While traditional Pakistani cuisine remains immensely beloved amongst citizens residing primarily in larger cosmopolitan centers nationwide inclusive even of bustling megacity hubs located right smack dapartment center amid sprawls stretching all directions alike—such as yours truly situated squarely nestled deep down southwest corner encompassing greater sind province’s largest conurbation colloquially referred to simply yet affectionately labeled “karachī,” homeowners craving international fare particularly italian style thin crispy slices loaded liberally onto chewy dough bases coined ‘pizza’, fortunately enjoy accessibility owing largely thanks mainly two prominent chains operating multiple branches scattered about different neighborhood zones namely domino‘s pepperoni garlic cheese stuffed supreme double treat california club veggie delight four cheese grand slamixt salad sticks dessert brownies cinnamon bitzzz drinks ice cream cookies milkshakes smoothies coffee tea juices soft 

In [38]:
import gradio as gr
messages = []

with gr.Blocks() as mychatbot:  # Blocks is a low-level API that allows
                                # you to create custom web applications
    chatbot = gr.Chatbot([], elem_id="NED SECC Chatbot V1.0")
    #chatbot = gr.Chatbot(height=680)      # displays a chatbot
    question = gr.Textbox()     # for user to ask a question
    clear = gr.Button("Clear Conversation")  # Clear button
    # function to clear the conversation
    def clear_messages():
        global messages, history
        messages = []    # reset the messages list
        memory.clear()

    def chat(message, chat_history):
        global messages
        messages.append({"role": "user", "content": message})
        response = chat(message)
        print(response)

        content = response#['choices'][0]['message']['content']
        messages.append({"role":"assistant", "content": content})

        chat_history.append((message, content))
        return "", chat_history

    # wire up the event handler for Submit button (when user press Enter)
    question.submit(fn = chat,
                    inputs = [question, chatbot],
                    outputs = [question, chatbot])

    # wire up the event handler for the Clear Conversation button
    clear.click(fn = clear_messages,
                inputs = None,
                outputs = chatbot,
                queue = False)

mychatbot.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://710b93406f7821b8d6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 495, in call_prediction
    output = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1561, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1179, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 807, in run
    re

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://710b93406f7821b8d6.gradio.live


In [ ]:
Attendance: